In [1]:
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
from shutil import copyfile
from shutil import move
import re

data_dir = 'D:/Downloads/amazon/'

df = pd.read_csv(data_dir + 'train.csv')
df.head(50)

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road
5,train_5,haze primary water
6,train_6,agriculture clear cultivation primary water
7,train_7,haze primary
8,train_8,agriculture clear cultivation primary
9,train_9,agriculture clear cultivation primary road


In [10]:
#test code
def pImageFileForIndex(index):
    source_filename = df.iloc[index]['image_name'] + '.jpg'
    #print(source_filename)
    #print(df.iloc[index]['tags'])

    words = df.iloc[index]['tags'].split()
    for curr_tag in words:
        if curr_tag == 'cloudy' or curr_tag == 'haze' or curr_tag == 'clear' or curr_tag == 'partly_cloudy':
            print(index, ' ', curr_tag)
            break

for index, row in df.iterrows():
    if (index > 10):
        break
    pImageFileForIndex(index)

(0, ' ', 'haze')
(1, ' ', 'clear')
(2, ' ', 'clear')
(3, ' ', 'clear')
(4, ' ', 'clear')
(5, ' ', 'haze')
(6, ' ', 'clear')
(7, ' ', 'haze')
(8, ' ', 'clear')
(9, ' ', 'clear')
(10, ' ', 'clear')


In [11]:
# copy images to the folder labeled with the right weather condition.  
# this is a preprocessing step for using Keras ImageDataGenerator.flow_from_directory.

preprocess_train_dir = data_dir + 'preprocess/train-jpg/'

def preprocessImageFileForIndex(index):
    source_filename = df.iloc[index]['image_name'] + '.jpg'
    #print(source_filename)
    #print(df.iloc[index]['tags'])

    data_src_dir = data_dir + 'train-jpg/'

    words = df.iloc[index]['tags'].split()
    for curr_tag in words:
        #print(curr_tag)
        if curr_tag == 'cloudy' or curr_tag == 'haze' or curr_tag == 'clear' or curr_tag == 'partly_cloudy':
            copyfile(data_src_dir + source_filename, preprocess_train_dir + curr_tag + '/'+ source_filename)
            break

In [13]:
def makedir(path):
    if not os.path.exists(path):
        os.makedirs(path)

makedir(preprocess_train_dir)

conditions = ['cloudy', 'haze', 'clear', 'partly_cloudy']
for item in conditions:
    makedir(preprocess_train_dir + item)

In [15]:
for index, row in df.iterrows():
    preprocessImageFileForIndex(index)


In [16]:
print('# File sizes')
for f in os.listdir(preprocess_train_dir):
    if not os.path.isdir(preprocess_train_dir + f):
        print(f.ljust(30) + str(round(os.path.getsize(preprocess_train_dir + f) / 1000000, 2)) + 'MB')
    else:
        sizes = [os.path.getsize(preprocess_train_dir+f+'/'+x)/1000000 for x in os.listdir(preprocess_train_dir + f)]
        print(f.ljust(30) + str(round(sum(sizes), 2)) + 'MB' + ' ({} files)'.format(len(sizes)))

# File sizes
clear                         0.0MB (28203 files)
cloudy                        0.0MB (2330 files)
haze                          0.0MB (2695 files)
partly_cloudy                 0.0MB (7251 files)


In [14]:
preprocess_validation_dir = data_dir + 'preprocess/validation-jpg/'

makedir(preprocess_validation_dir)

conditions = ['cloudy', 'haze', 'clear', 'partly_cloudy']
for item in conditions:
    makedir(preprocess_validation_dir + item)

In [17]:
import random
# Move random samples to validation-jpg folder
def randomSelectForValidationSet(validation_fraction, current_wx):
    print('working on : ' + current_wx)
    current_train_dir = preprocess_train_dir + current_wx + '/'
    current_validation_dir = preprocess_validation_dir  + current_wx + '/'
    list_files = os.listdir(current_train_dir)
    random.shuffle(list_files) # In-place shuffle
    validation_size = int(round(len(list_files) * validation_fraction, 0))
    print('validation_size:' , validation_size)
    validation_files = list_files[0:validation_size]
    for curr_file in validation_files:
        src = current_train_dir + curr_file
        dest = current_validation_dir + curr_file
        #print ('src: ' + src)
        #print ('dest: ' + dest)
        move(src, dest)
    

In [18]:
wx_list = ['cloudy', 'partly_cloudy', 'clear', 'haze']
for current_wx in wx_list:
    randomSelectForValidationSet(0.20, current_wx)  # 20% for validations

working on : cloudy
('validation_size:', 466)
working on : partly_cloudy
('validation_size:', 1450)
working on : clear
('validation_size:', 5641)
working on : haze
('validation_size:', 539)


In [19]:
print('# File sizes')
for f in os.listdir(preprocess_train_dir):
    if not os.path.isdir(preprocess_train_dir + f):
        print(f.ljust(30) + str(round(os.path.getsize(preprocess_train_dir + f) / 1000000, 2)) + 'MB')
    else:
        sizes = [os.path.getsize(preprocess_train_dir+f+'/'+x)/1000000 for x in os.listdir(preprocess_train_dir + f)]
        print(f.ljust(30) + str(round(sum(sizes), 2)) + 'MB' + ' ({} files)'.format(len(sizes)))

# File sizes
clear                         0.0MB (22562 files)
cloudy                        0.0MB (1864 files)
haze                          0.0MB (2156 files)
partly_cloudy                 0.0MB (5801 files)
